In [1]:
#importing necessary libraries
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
import math
import pickle
import joblib
import warnings
warnings.filterwarnings("ignore")
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', '{:.2f}'.format)

In [2]:
#Reading the data from files
calendar_df = pd.read_csv('calendar.csv')
sales_eval_df = pd.read_csv('sales_train_evaluation.csv')
prices_df = pd.read_csv('sell_prices.csv')

In [3]:
data_pt = sales_eval_df.sample(random_state=42)
data_pt = data_pt.reset_index(drop=True)
data_pt

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,FOODS_3_180_CA_1_evaluation,FOODS_3_180,FOODS_3,FOODS,CA_1,CA,0,0,0,0,...,0,0,1,0,0,1,0,0,0,0


In [4]:
def final_func_1(X):
    #here we are inserting the columns for the days d_1942 to d_1969 as nan for which we need to forecast sales
    for i in range(1942,1970):
        X['d_'+str(i)] = np.nan
        X['d_'+str(i)] = X['d_'+str(i)].astype(np.float16)
    
    #to transform the dataframe into vertical rows as each corresponds to each day sales of an item from a particular store
    X_melt = pd.melt(X, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],
                       var_name='d',value_name='sales')
    #creating a single dataframe
    X_melt = X_melt.merge(calendar_df,  on='d', how='left')
    X_melt = X_melt.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
    
    #pre processing missing values of prices by transforming with mean price of that id
    X_melt['sell_price'].fillna(X_melt.groupby('id')['sell_price'].transform('mean'),inplace=True)
    
    #creating lag features such that the for a product on current day it gets it's sales upto 3 months prior.
    shifting = 28 #shift period in order to account for 28 days to forecast
    for i in range(9): #num of weeks to shift here 8 weeks we consider
        X_melt['lag_'+str(shifting+(7*i))] = X_melt.groupby('id')['sales'].shift(shifting+(7*i)).astype(np.float16)
    
    #creating constant shift rolling agg features
    for i in [7,14,28,35,60]:
        X_melt['rolling_mean_'+str(i)] =  X_melt.groupby(['id'])['lag_28'].transform(lambda x: x.rolling(i).mean())
        X_melt['rolling_median_'+str(i)] =  X_melt.groupby(['id'])['lag_28'].transform(lambda x: x.rolling(i).median())
        
    #calender features
    X_melt['date'] = pd.to_datetime(X_melt['date'])
    #each day of the month
    X_melt['day_of_month'] = X_melt['date'].dt.day.astype(np.int8)
    #changing year value as 0 for 2011 and 1 for 2012 .... 5 for 2016
    X_melt['year'] = (X_melt['year'] - X_melt['year'].min()).astype(np.int8)
    #week number of a day in a month ex: 29th in January corresponds to 5th week of January
    X_melt['week_no_inmonth'] = X_melt['day_of_month'].apply(lambda x: math.ceil(x/7)).astype(np.int8)
    #checking if the day is weekend or not
    X_melt['is_weekend'] = (X_melt['wday']<=2).astype(np.int8)
    
    #changing the dtype to category for these columns in order to process the columns with label encoding
    cat_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2','snap_CA','snap_TX','snap_WI']
    lenc = LabelEncoder()
    for col in cat_cols:
        X_melt[col] = X_melt[col].astype('category')
        #preprocessing the categorical columns into label encoded columns
        X_melt[col] = lenc.fit_transform(X_melt[col].astype(str))
    #splitting the values of 'd' comlumn to take only the day number    
    X_melt['d'] = X_melt['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
    #final dataframe after pre-processing and feature engineering we are taking last 2 years historical sales
    X_melt = X_melt.loc[pd.to_datetime(X_melt['date'].dt.date) >= '2014-01-02']
    
    X_pre = X_melt.drop(['sales','date','weekday','wm_yr_wk'],axis=1)
    X_pre.reset_index(drop=True,inplace=True)
    n_rows = int(len(X_pre)*0.2)
    X_pre  = X_pre.tail(n_rows)
    N = 18
    X_pre_pred = pd.DataFrame()
    predictions_df = pd.DataFrame()
    best_base_models = ['M'+str(i) for i in range(1,N+1)]
    preds_X_pre  = ['pred_'+str(i) for i in range(int(X_pre.iloc[0]['d']),int(X_pre.iloc[-1]['d'])+1)]
    features_X_pre_pred = ['X_fea_'+str(i) for i in range(1,N+1)]
    #N represents number of base models
    for i in tqdm(range(N)):
        #predicting for all the days of X_pre using trained N base models and using them as features
        preds_X_pre[i] = pd.DataFrame()
        file_name = best_base_models[i]+'.pkl'
        for k in range(int(X_pre.iloc[0]['d']),int(X_pre.iloc[-1]['d'])+1):
            best_base_models[i] = joblib.load(file_name)
            best_base_models[i].n_jobs = -1
            preds_X_pre[i]['d_'+str(k)] = best_base_models[i].predict(X_pre[X_pre['d']==k])
        df1 = pd.melt(preds_X_pre[i],var_name='d',value_name='sales')
        X_pre_pred[features_X_pre_pred[i]] = df1['sales'].values
    best_metaM = joblib.load('best_meta_model.pkl')
    best_metaM.n_jobs = -1
    predictions = best_metaM.predict(X_pre_pred.values)
    #slicing the predictions such that to get each day predictions of all the products of test data
    start = 0
    t = int(X_pre.iloc[0]['d'])
    while start < len(predictions):
        end = start + 1
        predictions_df['d_'+str(t)] = predictions[start:end]
        start = end
        t = t+1
    predictions_df = pd.concat([X['id'],predictions_df],axis=1,sort=False)
    predictions_df_val = predictions_df[['id']]
    #validation predictions from days 1914-1941
    for i in range(28):
        predictions_df_val['F'+str(i+1)] = predictions_df['d_'+str(1914+i)]
    predictions_df_val['id'] =  predictions_df_val['id'].apply(lambda x: x.replace('evaluation','validation'))
    predictions_df_eval = predictions_df_val.copy()
    #evaluation predictions from days 1942-1969
    for i in range(28):
        predictions_df_eval['F'+str(i+1)] = predictions_df['d_'+str(1942+i)]
    predictions_df_eval["id"] = predictions_df_eval["id"].apply(lambda x: x.replace('validation','evaluation'))
    final_predictions = predictions_df_val.append(predictions_df_eval).reset_index(drop=True)
    return final_predictions

In [5]:
forecasted_sales = final_func_1(data_pt)

100%|██████████| 18/18 [09:20<00:00, 31.11s/it]


In [6]:
print('Forecast sales from days 1914 till 1941 is:')
forecasted_sales.iloc[[0]]

Forecast sales from days 1914 till 1941 is:


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_3_180_CA_1_validation,0.58,0.52,0.51,0.44,0.47,0.60,0.58,0.43,0.41,...,0.39,0.44,0.49,0.49,0.44,0.44,0.43,0.43,0.54,0.52


In [7]:
print('Forecast sales from days 1942 till 1969 is:')
forecasted_sales.iloc[[1]]

Forecast sales from days 1942 till 1969 is:


,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
1,FOODS_3_180_CA_1_evaluation,0.43,0.40,0.46,0.40,0.49,0.53,0.67,0.45,0.48,...,0.49,0.61,0.63,0.48,0.47,0.50,0.45,0.47,0.57,0.55


In [4]:
def final_func_2(X,y):
    #here we are inserting the columns for the days d_1914 to d_1941 as nan for which we need to forecast sales
    for i in range(1914,1942):
        X['d_'+str(i)] = np.nan
        X['d_'+str(i)] = X['d_'+str(i)].astype(np.float16)
    
    #to transform the dataframe into vertical rows as each corresponds to each day sales of an item from a particular store
    X_melt = pd.melt(X, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],
                       var_name='d',value_name='sales')
    #creating a single dataframe
    X_melt = X_melt.merge(calendar_df,  on='d', how='left')
    X_melt = X_melt.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
    
    #pre processing missing values of prices by transforming with mean price of that id
    X_melt['sell_price'].fillna(X_melt.groupby('id')['sell_price'].transform('mean'),inplace=True)
    
    #creating lag features such that the for a product on current day it gets it's sales upto 3 months prior.
    shifting = 28 #shift period in order to account for 28 days to forecast
    for i in range(9): #num of weeks to shift here 8 weeks we consider
        X_melt['lag_'+str(shifting+(7*i))] = X_melt.groupby('id')['sales'].shift(shifting+(7*i)).astype(np.float16)
    
    #creating constant shift rolling agg features
    for i in [7,14,28,35,60]:
        X_melt['rolling_mean_'+str(i)] =  X_melt.groupby(['id'])['lag_28'].transform(lambda x: x.rolling(i).mean())
        X_melt['rolling_median_'+str(i)] =  X_melt.groupby(['id'])['lag_28'].transform(lambda x: x.rolling(i).median())
        
    #calender features
    X_melt['date'] = pd.to_datetime(X_melt['date'])
    #each day of the month
    X_melt['day_of_month'] = X_melt['date'].dt.day.astype(np.int8)
    #changing year value as 0 for 2011 and 1 for 2012 .... 5 for 2016
    X_melt['year'] = (X_melt['year'] - X_melt['year'].min()).astype(np.int8)
    #week number of a day in a month ex: 29th in January corresponds to 5th week of January
    X_melt['week_no_inmonth'] = X_melt['day_of_month'].apply(lambda x: math.ceil(x/7)).astype(np.int8)
    #checking if the day is weekend or not
    X_melt['is_weekend'] = (X_melt['wday']<=2).astype(np.int8)
    
    #changing the dtype to category for these columns in order to process the columns with label encoding
    cat_cols = ['id','item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2','snap_CA','snap_TX','snap_WI']
    lenc = LabelEncoder()
    for col in cat_cols:
        X_melt[col] = X_melt[col].astype('category')
        #preprocessing the categorical columns into label encoded columns
        X_melt[col] = lenc.fit_transform(X_melt[col].astype(str))
    #splitting the values of 'd' comlumn to take only the day number    
    X_melt['d'] = X_melt['d'].apply(lambda x: x.split('_')[1]).astype(np.int16)
    #final dataframe after pre-processing and feature engineering we are taking last 2 years historical sales
    X_melt = X_melt.loc[pd.to_datetime(X_melt['date'].dt.date) >= '2014-01-02']
    
    X_pre = X_melt.drop(['sales','date','weekday','wm_yr_wk'],axis=1)
    X_pre.reset_index(drop=True,inplace=True)
    n_rows = int(len(X_pre)*0.2)
    X_pre  = X_pre.tail(n_rows)
    N = 18
    X_pre_pred = pd.DataFrame()
    predictions_df = pd.DataFrame()
    best_base_models = ['M'+str(i) for i in range(1,N+1)]
    preds_X_pre  = ['pred_'+str(i) for i in range(int(X_pre.iloc[0]['d']),int(X_pre.iloc[-1]['d'])+1)]
    features_X_pre_pred = ['X_fea_'+str(i) for i in range(1,N+1)]
    #N represents number of base models
    for i in tqdm(range(N)):
        #predicting for all the days of X_pre using trained N base models and using them as features
        preds_X_pre[i] = pd.DataFrame()
        file_name = best_base_models[i]+'.pkl'
        for k in range(int(X_pre.iloc[0]['d']),int(X_pre.iloc[-1]['d'])+1):
            best_base_models[i] = joblib.load(file_name)
            best_base_models[i].n_jobs = -1
            preds_X_pre[i]['d_'+str(k)] = best_base_models[i].predict(X_pre[X_pre['d']==k])
        df1 = pd.melt(preds_X_pre[i],var_name='d',value_name='sales')
        X_pre_pred[features_X_pre_pred[i]] = df1['sales'].values
    best_metaM = joblib.load('best_meta_model.pkl')
    best_metaM.n_jobs = -1
    predictions = best_metaM.predict(X_pre_pred.values)
    #slicing the predictions such that to get each day predictions of all the products of test data
    start = 0
    t = int(X_pre.iloc[0]['d'])
    while start < len(predictions):
        end = start + 1
        predictions_df['d_'+str(t)] = predictions[start:end]
        start = end
        t = t+1
    predictions_df = pd.concat([X['id'],predictions_df],axis=1,sort=False)
    predictions_df_val = predictions_df[['id']]
    #validation predictions from days 1914-1941
    for i in range(28):
        predictions_df_val['F'+str(i+1)] = predictions_df['d_'+str(1914+i)]
    predictions_df_val['id'] =  predictions_df_val['id'].apply(lambda x: x.replace('evaluation','validation'))
    
    df = X.loc[:,'d_1070':'d_1941']
    train_df = df.iloc[:,:-28]
    scale_lst = []
    for i in range(len(train_df)):
        val = train_df.iloc[i].values
        # to consider the periods following the first non-zero demand observed for the series under evaluation.
        val = val[np.argmax(val!=0):]
        #to scale the squared-error as taking the consecutive difference of each day sales
        scale = ((val[1:] - val[:-1]) ** 2).mean()
        #storing the scale value corresponding to each time series
        scale_lst.append(scale)
    scale_arr = np.array(scale_lst)
    predictions_df_val.drop(['id'],inplace=True,axis=1)
    predictions_df_val.columns = ['d_'+str(1914+i) for i in range(28)]
    #computing mean squared error
    num = ((predictions_df_val - y)**2).mean(axis=1)
    #scaled error i.e., root mean squared scaled error
    rmsse = (num/scale_arr).map(np.sqrt)
    #since we have a single time series the weight in WRMSSE is 1
    w_i = 1
    WRMSSE = w_i*rmsse
    return WRMSSE

In [5]:
WRMSSE = final_func_2(data_pt.iloc[:,:-28],data_pt.iloc[:,-28:])

100%|██████████| 18/18 [08:42<00:00, 29.03s/it]


In [7]:
print('The evaluated error metric WRMSSE is: ',round(np.float(WRMSSE.values),3))

The evaluated error metric WRMSSE is:  0.775
